# Switching Assistant
- Intent classification and Service Routing
- The LLM analyzes this combination and classifies the input into one or combination of three main intents: AppAssistant, PainAssistant, or PersonalAssistant.
- Creating through bedrock-agent for more confirugation and control

In [1]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.34


--- 

In [2]:
# Bedrock Agent runtime
pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name = region)

In [3]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [4]:
def prompt_choice (prompt_template1, query, contexts, choice, prompt_template2 = None):

        if (choice == 'Intent'):
            prompt = f"""
            Human: Use the following <context> to determine if the intent of <query> is classified as AI App assistant or as the 
            Pain assistant as decribed in <prompt_template1>. 

            If the intent is classified as AI App assistant then respond with 'AI Assistant for the App', 
            if the intent is classified as the Pain assistant then respond with 'Pain Psychologist',
            if the intent is classifed as both the AI App assistant and Pain assistant then respond with 'AI Assistant for the App, Pain Psychologist'
            
            <prompt_template1>
            {prompt_template1}
            </prompt_template1>
            
            <context>
            {contexts}
            </context>
            
            <query>
            {query}
            </query>
        
            Only respond with the title name of the assistant(s) chosen. 
            Do not respond if the prompt is not related to <contexts>.
            
            Assistant:"""
            
        elif (choice == 'Pain'):
            prompt = f"""
            Human: Use the following <context> to answer the <query> according to your role in <prompt_template1>. 
            
            
            <prompt_template1>
            {prompt_template1}
            </prompt_template1>
            
            <context>
            {contexts}
            </context>
            
            <query>
            {query}
            </query>

            Do not self-reference yourself as an Pain pscyhologist.
            Do not respond if the prompt is not related to pain Psychology.
            Do not respond if the prompt is not related to <contexts>.
            Do not respond if the prompt is related to usage of the app.
            
            Assistant:"""


        elif (choice == 'App'):
            prompt = f"""
            Human: Use the following <context> to answer the <query> according to your role in <prompt_template1>. 
            
            <prompt_template1>
            {prompt_template1}
            </prompt_template1>
            
            <context>
            {contexts}
            </context>
            
            <query>
            {query}
            </query>

            Do not self-reference yourself as an AI assistant.
            Do not respond if the prompt is not related to the app.
            
            Assistant:"""

        elif (choice == 'Both'):
            prompt = f"""
            Human: Your role is <prompt_template1> and <prompt_template2>. Please combine the texts in <query> to make it a consistent 
            and remove any iterations. Be concise when responding.
            
            
            <prompt_template1>
            {prompt_template1}
            </prompt_template1>
            
            <prompt_template2>
            {prompt_template2}
            </prompt_template2>
            
            <query>
            {query}
            </query>
            
            Assistant:"""


        return (prompt)


def prompt_template_choice(choice):
    # Assigning the default knowledge base prompt
    path = '../machine-learning/4. Knowledge Base Template/'

    if (choice == 'Intent'):
        filename = 'Switching_Assistant_V1_Oct17.md'

    elif (choice == 'Pain'):
        filename = 'Pain_Assistant_V1_Oct15.md'
            
    elif (choice == 'App'):
        filename = 'App_Assistant_V1_Oct10.md'
        
    with open(path+filename, 'r') as f:
        prompt_template = f.read()
            
    return (prompt_template)



In [5]:
def data_ingestion (prompt, prompt_template, contexts, query, temp = 1):

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": temp,
    "top_p": 1
        }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType,
                                           guardrailIdentifier = 'o7v7nzog6nch',
                                           guardrailVersion = 'DRAFT')
    
    return (response)

In [6]:
# Knowledge base ID

kb_aws_bedorck_s3 = '76UIT87ACB'
kb_app_assistant = 'V2W0LT3GZP'
kb_pain_assistant = 'KESDTCXEJE'

--- 
# Role switching 

In [11]:
############ Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/Role Switching/'
filename = 'Role-Switching2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

In [8]:
############ Main body of the analysis

for query in queries:
    
    print ('------------------------------------------------------------------------')
    
    # Acquire the context
    response = retrieve(query, kb_aws_bedorck_s3)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)
    
    # Choose the prompt template
    prompt_template_intent = prompt_template_choice('Intent')
    
    # Format the Prompt with the prompt template
    prompt_role_intent = prompt_choice (prompt_template_intent, query, contexts, 'Intent')
    
    # Ingest data into the LLM (Modifications can be applied here)
    response = data_ingestion (prompt_role_intent, prompt_template_intent, contexts, query, 0)
    
    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text_intent = response_body.get('content')[0]['text']
    
    if ((response_text_intent == 'AI Assistant for the App') or (response_text_intent == 'AI Assistant for the App, Pain Psychologist')):
        
        # Acquire the context
        response = retrieve(query, kb_app_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template_app = prompt_template_choice('App')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template_app, query, contexts, 'App')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template_app, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_app = response_body.get('content')[0]['text']    
    
    if ((response_text_intent == 'Pain Psychologist') or (response_text_intent == 'AI Assistant for the App, Pain Psychologist')):
    
        # Acquire the context
        response = retrieve(query, kb_pain_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template_pain = prompt_template_choice('Pain')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template_pain, query, contexts, 'Pain')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template_pain, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_pain = response_body.get('content')[0]['text']
    
    if (response_text_intent == 'AI Assistant for the App, Pain Psychologist'):
    
        # Acquire the context
        
        combined_query = response_text_pain + ' ' + response_text_app
        
        response = retrieve(combined_query, kb_aws_bedorck_s3)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
    
        # Choose the prompt template
        prompt_template_pain_and_app = prompt_template_pain + ' ' + prompt_template_app
    
        # Format the Prompt with the prompt template - This combines both the templates
        prompt_role_pain_and_app = prompt_choice (prompt_template_pain, query, contexts, 'Both', prompt_template_app)
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_pain_and_app, prompt_template_pain_and_app, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_pain_and_app = response_body.get('content')[0]['text']
    
    ############ Print the output
    print (query)
    print ('\n')
    
    if (response_text_intent == 'AI Assistant for the App, Pain Psychologist'):
        print (response_text_pain_and_app)
    
    elif (response_text_intent == 'Pain Psychologist'):
        print (response_text_pain)
    
    elif (response_text_intent == 'AI Assistant for the App'):
        print (response_text_app)
        
    
print ('------------------------------------------------------------------------')


------------------------------------------------------------------------
How do I update my pain condition in the app? How does stress or anxiety affect my pain levels?


To update your pain condition in the Manage My Pain app, go to the Settings section and select "Edit Conditions." From there, you can update your pain condition details or add new conditions if needed.

Regarding stress and anxiety affecting pain levels, it's a common issue. Stress and anxiety can exacerbate chronic pain by increasing muscle tension, triggering inflammation, and amplifying pain signals in the body. Managing stress through relaxation techniques, counseling, or mindfulness practices can often help reduce pain intensity. Do you want to learn more about coping strategies for dealing with stress-related pain?
------------------------------------------------------------------------
Can I customize the frequency of notifications?


Yes, you can customize the frequency of notifications in the Manage My Pain a

# Dr. Max Questions

In [13]:
############ Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/Pain Assistant/'
filename = 'Pain-Assistant-Max-Questions1.md'
# filename = 'Role-Switching1.md'
# filename = 'Role-Switching2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

In [10]:

############ Main body of the analysis

for query in queries:
    
    print ('------------------------------------------------------------------------')
    
    # Acquire the context
    response = retrieve(query, kb_aws_bedorck_s3)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)
    
    # Choose the prompt template
    prompt_template_intent = prompt_template_choice('Intent')
    
    # Format the Prompt with the prompt template
    prompt_role_intent = prompt_choice (prompt_template_intent, query, contexts, 'Intent')
    
    # Ingest data into the LLM (Modifications can be applied here)
    response = data_ingestion (prompt_role_intent, prompt_template_intent, contexts, query, 0)
    
    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text_intent = response_body.get('content')[0]['text']
    
    if ((response_text_intent == 'AI Assistant for the App') or (response_text_intent == 'AI Assistant for the App, Pain Psychologist')):
        
        # Acquire the context
        response = retrieve(query, kb_app_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template_app = prompt_template_choice('App')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template_app, query, contexts, 'App')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template_app, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_app = response_body.get('content')[0]['text']    
    
    if ((response_text_intent == 'Pain Psychologist') or (response_text_intent == 'AI Assistant for the App, Pain Psychologist')):
    
        # Acquire the context
        response = retrieve(query, kb_pain_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template_pain = prompt_template_choice('Pain')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template_pain, query, contexts, 'Pain')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template_pain, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_pain = response_body.get('content')[0]['text']
    
    if (response_text_intent == 'AI Assistant for the App, Pain Psychologist'):
    
        # Acquire the context
        
        combined_query = response_text_pain + ' ' + response_text_app
        
        response = retrieve(combined_query, kb_aws_bedorck_s3)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
    
        # Choose the prompt template
        prompt_template_pain_and_app = prompt_template_pain + ' ' + prompt_template_app
    
        # Format the Prompt with the prompt template - This combines both the templates
        prompt_role_pain_and_app = prompt_choice (prompt_template_pain, query, contexts, 'Both', prompt_template_app)
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_pain_and_app, prompt_template_pain_and_app, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_pain_and_app = response_body.get('content')[0]['text']
    
    ############ Print the output
    print (query)
    print ('\n')
    # print (response_text_intent)
    # print ('\n')
    
    if (response_text_intent == 'AI Assistant for the App, Pain Psychologist'):
        print (response_text_pain_and_app)
    
    elif (response_text_intent == 'Pain Psychologist'):
        print (response_text_pain)
    
    elif (response_text_intent == 'AI Assistant for the App'):
        print (response_text_app)
        
    
print ('------------------------------------------------------------------------')


------------------------------------------------------------------------
What is pain? 


Pain is a complex experience that involves both physical and psychological components. From a psychological perspective, pain can be influenced by factors such as thoughts, emotions, beliefs, and coping strategies. Here are some key points about understanding pain:

- Pain is a subjective experience, and two people with the same injury or condition may experience different levels of pain.
- Chronic pain can lead to negative emotional states like anxiety, depression, and hopelessness, which can further exacerbate the pain experience.
- Cognitive-behavioral techniques, such as cognitive restructuring, mindfulness, and relaxation strategies, can help manage the psychological aspects of pain.
- Accepting that pain is a part of life and focusing on what you can control can help reduce suffering and improve quality of life.

Would you like me to expand on any specific psychological approaches or coping 

# Testing specific Queries

In [17]:
queries = ['How do I update my pain condition in the app?']

for query in queries:
    response = retrieve(query, kb_aws_bedorck_s3)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

contexts


["# Getting Started ## 1. Personalize the Application ### Can I personalize the application without logging in to the app? Yes, you can personalize the application without logging in to the app. ### Do I need to accept the End User License Agreement? Yes, accepting the End User License Agreement is required to use Manage My Pain. ### Why can't I find my pain condition? Our users have added thousands of conditions, but we just show the most common.  If you can't find one that suits your situation, simply add it! ### Do I have to add any pain conditions? No, all information entered into Manage My Pain is voluntary.  Keep in mind, the more information that is entered, the more powerful the reports can be. ### Why are you asking me this? This information allows us to configure the application to better suit your situation, and ensures that the reports generated from Manage My Pain are a better communication tool. ### Do I have to add medications? No, all information entered into Manage My 

In [16]:

############ Main body of the analysis

for query in queries:
    
    print ('------------------------------------------------------------------------')
    
    # Acquire the context
    response = retrieve(query, kb_aws_bedorck_s3)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)
    
    # Choose the prompt template
    prompt_template_intent = prompt_template_choice('Intent')
    
    # Format the Prompt with the prompt template
    prompt_role_intent = prompt_choice (prompt_template_intent, query, contexts, 'Intent')
    
    # Ingest data into the LLM (Modifications can be applied here)
    response = data_ingestion (prompt_role_intent, prompt_template_intent, contexts, query, 0)
    
    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text_intent = response_body.get('content')[0]['text']
    
    if ((response_text_intent == 'AI Assistant for the App') or (response_text_intent == 'AI Assistant for the App, Pain Psychologist')):
        
        # Acquire the context
        response = retrieve(query, kb_app_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template_app = prompt_template_choice('App')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template_app, query, contexts, 'App')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template_app, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_app = response_body.get('content')[0]['text']    
    
    if ((response_text_intent == 'Pain Psychologist') or (response_text_intent == 'AI Assistant for the App, Pain Psychologist')):
    
        # Acquire the context
        response = retrieve(query, kb_pain_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template_pain = prompt_template_choice('Pain')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template_pain, query, contexts, 'Pain')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template_pain, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_pain = response_body.get('content')[0]['text']
    
    if (response_text_intent == 'AI Assistant for the App, Pain Psychologist'):
    
        # Acquire the context
        
        combined_query = response_text_pain + ' ' + response_text_app
        
        response = retrieve(combined_query, kb_aws_bedorck_s3)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
    
        # Choose the prompt template
        prompt_template_pain_and_app = prompt_template_pain + ' ' + prompt_template_app
    
        # Format the Prompt with the prompt template - This combines both the templates
        prompt_role_pain_and_app = prompt_choice (prompt_template_pain, query, contexts, 'Both', prompt_template_app)
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_pain_and_app, prompt_template_pain_and_app, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text_pain_and_app = response_body.get('content')[0]['text']
    
    ############ Print the output
    print (query)
    print ('\n')
    # print (response_text_intent)
    # print ('\n')
    
    if (response_text_intent == 'AI Assistant for the App, Pain Psychologist'):
        print (response_text_pain_and_app)
    
    elif (response_text_intent == 'Pain Psychologist'):
        print (response_text_pain)
    
    elif (response_text_intent == 'AI Assistant for the App'):
        print (response_text_app)
        
    
print ('------------------------------------------------------------------------')


------------------------------------------------------------------------
How do I update my pain condition in the app?


To update your pain condition in the Manage My Pain app, you can follow these steps:

1. Open the app and go to the "Profile" or "Settings" section.
2. Look for an option related to "Pain Conditions" or "Health Conditions".
3. Select the pain condition you want to update.
4. Edit the details or information about that condition as needed.
5. Save the changes.

The app allows you to customize and update the details of your pain conditions to ensure the data accurately reflects your current situation. Just navigate to the appropriate section within your profile settings.

Do you want me to provide more details on updating pain conditions or managing your profile in the app?
------------------------------------------------------------------------
